In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sqlite3

In [2]:
competitor_referrals = pd.read_csv('../data/nashville_referrals.csv')

In [3]:
competitor_referrals.head()

,index,from_npi,to_npi,patient_count,transaction_count,average_day_wait,std_day_wait,from_npi_specialty,to_npi_specialty
0,705,1013154723,1003028770,32,50,41.940,52.320,Durable Medical Equipment & Medical Supplies,Dermatology
1,706,1003963976,1003028770,2535,3945,0.000,0.000,Dermatology,Dermatology
2,1139,1003854258,1003052697,31,55,20.436,26.403,Family Medicine,Home Health
3,1713,1043232879,1003076233,76,78,28.244,52.860,Radiology,Internal Medicine
4,1755,1023046901,1003077728,61,67,20.448,54.516,Radiology,Thoracic Surgery (Cardiothoracic Vascular Surg...


In [4]:
#removing extra index column

competitor_referrals = competitor_referrals.drop(columns='index')

In [5]:
competitor_referrals['to_npi_specialty'].value_counts()

Internal Medicine                     44796
Radiology                             21417
General Acute Care Hospital           13879
Clinical Medical Laboratory           10198
Pathology                              7727
                                      ...  
Point of Service                          1
Prevention Professional                   1
Hospice, Inpatient                        1
Early Intervention Provider Agency        1
Electrodiagnostic Medicine                1
Name: to_npi_specialty, Length: 99, dtype: int64

In [6]:
competitor_referrals = competitor_referrals[competitor_referrals['to_npi_specialty'] == 'General Acute Care Hospital']

In [7]:
db = sqlite3.connect('../data/hop_teaming.sqlite')

query = '''
SELECT npi, provider_organization_name, provider_first_name, provider_last_name, city, zip_5
FROM nppes
'''

npi_orgname = pd.read_sql(query, db)

In [8]:
competitor_referrals = competitor_referrals.merge(npi_orgname, how='left', left_on='to_npi', right_on='npi')

In [9]:
competitor_referrals = competitor_referrals.rename(columns={'provider_organization_name': 'to_org_name', 
                                     'provider_first_name':'to_fname', 
                                     'provider_last_name':'to_lname',
                                     'city':'to_city',
                                     'zip_5':'to_zip_5'})

In [10]:
competitor_referrals = competitor_referrals.drop(columns = 'npi')

In [11]:
competitor_referrals = competitor_referrals.merge(npi_orgname, how = 'left', left_on='from_npi', right_on = 'npi')

In [12]:
competitor_referrals = competitor_referrals.rename(columns={'provider_organization_name': 'from_org_name', 
                                     'provider_first_name':'from_fname', 
                                     'provider_last_name':'from_lname',
                                     'city':'from_city',
                                     'zip_5':'from_zip_5'})

In [13]:
competitor_referrals.head()

,from_npi,to_npi,patient_count,transaction_count,average_day_wait,std_day_wait,from_npi_specialty,to_npi_specialty,to_org_name,to_fname,to_lname,to_city,to_zip_5,npi,from_org_name,from_fname,from_lname,from_city,from_zip_5
0,1023336872,1003996000,75,78,10.923,37.348,Pathology,General Acute Care Hospital,THE HEALTH CARE AUTHORITY OF THE CITY OF ANNISTON,None,None,ANNISTON,36207,1023336872,None,EVAN,ALSTON,MURFREESBORO,37129
1,1033570015,1023055126,63,72,26.722,44.023,Nurse Practitioner,General Acute Care Hospital,"HCA HEALTH SERVICES OF TENNESSEE, INC.",None,None,NASHVILLE,37203,1033570015,None,LESLEY,BALLANCE,NASHVILLE,37203
2,1033532585,1023055126,152,214,40.439,59.850,Nurse Practitioner,General Acute Care Hospital,"HCA HEALTH SERVICES OF TENNESSEE, INC.",None,None,NASHVILLE,37203,1033532585,None,DENISE,BROWN,NASHVILLE,37203
3,1033537576,1023055126,1357,1666,9.681,35.407,Ambulance,General Acute Care Hospital,"HCA HEALTH SERVICES OF TENNESSEE, INC.",None,None,NASHVILLE,37203,1033537576,LIFEGUARD AMBULANCE SERVICE LLC,None,None,NASHVILLE,37210
4,1043310428,1023055126,409,578,32.955,42.327,Radiology,General Acute Care Hospital,"HCA HEALTH SERVICES OF TENNESSEE, INC.",None,None,NASHVILLE,37203,1043310428,"TENNESSEE ONCOLOGY,PLLC",None,None,NASHVILLE,37203
